In [1]:
import os
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [4]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5), # 3 for RGB Channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],    # 1 for greyscale channels
                         std=[0.5])
])

In [6]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True)

In [8]:
# Descriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

# Generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

D = D.to(device)
G = G.to(device)

In [9]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [10]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [11]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [12]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # Train the descriminator
        # Compute BCE_Loss using real images where BCE_Loss(x, y): -y * log(D(x)) - (1 - y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCE_Loss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Train the generator
        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z))) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}, D(X): {real_score.mean().item():.2f}, D(G(z)): {fake_score.mean().item():.2f}')
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, f'fake_images-{epoch+1}.png'))
        

Epoch [1/0], Step [200/600], d_loss: 0.0400, g_loss: 4.1854, D(X): 1.00, D(G(z)): 0.03531898558139801
Epoch [1/0], Step [400/600], d_loss: 0.0763, g_loss: 5.8085, D(X): 0.98, D(G(z)): 0.04789315164089203
Epoch [1/0], Step [600/600], d_loss: 0.0375, g_loss: 4.9378, D(X): 0.99, D(G(z)): 0.022466957569122314
Epoch [2/1], Step [200/600], d_loss: 0.0586, g_loss: 4.6159, D(X): 0.99, D(G(z)): 0.04860576614737511
Epoch [2/1], Step [400/600], d_loss: 0.1172, g_loss: 3.7184, D(X): 0.99, D(G(z)): 0.0957982987165451
Epoch [2/1], Step [600/600], d_loss: 0.3185, g_loss: 6.7508, D(X): 0.87, D(G(z)): 0.014630336314439774
Epoch [3/2], Step [200/600], d_loss: 0.2857, g_loss: 3.8156, D(X): 0.96, D(G(z)): 0.19043022394180298
Epoch [3/2], Step [400/600], d_loss: 0.4983, g_loss: 5.6985, D(X): 0.82, D(G(z)): 0.1033599004149437
Epoch [3/2], Step [600/600], d_loss: 0.1714, g_loss: 4.7798, D(X): 0.94, D(G(z)): 0.08019533008337021
Epoch [4/3], Step [200/600], d_loss: 0.0794, g_loss: 4.5766, D(X): 0.96, D(G(z)): 

In [13]:
# Save the model checkpoints
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')